In [1]:
pip install "numpy<2.0" 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import os
import keras
import shutil
import numpy as np
from mtcnn import MTCNN
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
path = 'Real_Data/Extracted/'

In [ ]:
dir_ = os.listdir('./Real_Data/Extracted/')

In [39]:
for i in dir_:
    a = i + "/video"
    print(os.listdir(path + i +'/video')[3:])

['sa1', 'sa2', 'si1279', 'si1909', 'si649', 'sx109', 'sx19', 'sx199', 'sx289', 'sx379']
['sa1', 'sa2', 'si1573', 'si2203', 'si943', 'sx133', 'sx223', 'sx313', 'sx403', 'sx43']
['sa1', 'sa2', 'si1178', 'si1808', 'si548', 'sx188', 'sx278', 'sx368', 'sx8', 'sx98']
['sa1', 'sa2', 'si1454', 'si2084', 'si824', 'sx104', 'sx14', 'sx194', 'sx284', 'sx374']
['sa1', 'sa2', 'si1474', 'si2104', 'si844', 'sx124', 'sx214', 'sx304', 'sx34', 'sx394']
['sa1', 'sa2', 'si1544', 'si1566', 'si2149', 'sx104', 'sx14', 'sx194', 'sx284', 'sx374']
['sa1', 'sa2', 'si1084', 'si1653', 'si1714', 'sx184', 'sx274', 'sx364', 'sx4', 'sx94']
['sa1', 'sa2', 'si1386', 'si2016', 'si756', 'sx126', 'sx216', 'sx306', 'sx36', 'sx396']
['sa1', 'sa2', 'si1400', 'si2030', 'si770', 'sx140', 'sx230', 'sx320', 'sx410', 'sx50']
['sa1', 'sa2', 'si1264', 'si1894', 'si634', 'sx184', 'sx274', 'sx364', 'sx4', 'sx94']
['sa1', 'sa2', 'si1116', 'si1587', 'si1746', 'sx126', 'sx216', 'sx306', 'sx36', 'sx396']
['sa1', 'sa2', 'si1265', 'si635', '

In [18]:
for i in os.listdir('Real_Data/Extracted/fadg0/video/sa1'):
    name = i + '.jpg'
    os.rename(path + i, name)

In [37]:
subject_ids = os.listdir(path)

In [ ]:
subject_ids

In [40]:
for subject in subject_ids:
    new_path = path + subject + '/video/'
    categories = os.listdir(new_path)[3:]
    for cat in categories:
        frames = os.listdir(new_path + cat)
        for frame in frames:
            frame_path = new_path + cat + '/'
            os.rename(frame_path + frame, frame_path+frame + '.jpg')

In [ ]:
# Function to convert the frames to a video

def write_video_from_frames(frame_folder, output_video_path, fps=30):
    images = sorted(os.listdir(frame_folder))
    first_frame = cv2.imread(os.path.join(frame_folder, images[0]))

    if first_frame is None:
        raise ValueError("No valid frames found!")

    height, width, _ = first_frame.shape


    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image in images:
        frame = cv2.imread(os.path.join(frame_folder, image))
        if frame is not None:
            out.write(frame)  

    out.release()
    print(f"Video saved at: {output_video_path}")

In [ ]:
real_dir = os.listdir("./Real_Data/Extracted/")


In [ ]:
path = "./Real_Data/Extracted/"

In [ ]:
for subject in real_dir:
    os.makedirs('./Real_Data_Video/' + subject, exist_ok=True)
    sub_path = path + subject + '/video/'
    for category in os.listdir(sub_path)[3:]:
        category_path = './Real_Data_Video/' + subject + '/' + category
        write_video_from_frames(sub_path + category,category_path+".mp4",fps=25)


## ***Detecting Cordinates Of Face In Each Frame By MTCNN and Converting Videos Of Fake Dataset Into Compressed .npz***

In [7]:
fake_data_dir = os.listdir('Fake_Data/higher_quality')

In [8]:
path = 'Fake_Data/higher_quality/'

In [4]:
detector = MTCNN()

In [45]:
# Reading a video frame by frame and detecting cordinates of the face. Returning a numpy array of size (Total number of frames,244,244,3)

def video_to_numpy(path):
    detector = MTCNN(device = "GPU:0")
    array_list = []
    cap = cv2.VideoCapture(path)
    if cap.isOpened() == False:
        raise RuntimeError

    while cap.isOpened():
        ret, frame = cap.read()
    
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face = detector.detect_faces(frame)
            if not face :
                continue
            col_1, row_1, col_2, row_2 = face[0]['box']
            col_1_, row_1_, col_2_, row_2_ = col_1, row_1, col_2 + col_1, row_2 + row_1
            frame = frame[row_1_ : row_2_, col_1_ : col_2_]
            
            if frame is None or frame.size == 0:
                continue
                
            frame = cv2.resize(frame, dsize = (224,224))
            array_list.append(frame)
    
        else: 
            break

    cap.release()
            
    return np.array(array_list)

In [ ]:
real_data_dir = os.listdir('Real_Data_Video')

In [ ]:
path = 'Real_Data_Video/'

In [ ]:
for subject in real_data_dir:
    
    os.makedirs('./Real_Data_Video_Numpy/'+ subject, exist_ok=True)
    sub_path = path + subject

    for video in os.listdir(sub_path):
        array = video_to_numpy(sub_path + '/' + video)
        video_path = "./Real_Data_Video_Numpy/"+ subject + '/' + video
        np.savez_compressed(video_path, array)
        print(video_path)


In [ ]:
for subject in fake_data_dir[9:]:
    
    os.makedirs('./Fake_Data_Numpy/'+ subject, exist_ok=True)
    sub_path = path + subject

    for video in os.listdir(sub_path):
        if video.endswith('.avi'):
            array = video_to_numpy(sub_path + '/' + video)
            video_path = './Fake_Data_Numpy/'+ subject + '/' + video.split(sep = '-')[0]
            np.savez_compressed(video_path, array)
            print(video_path)


## ***Model building and training***

In [4]:
def data_gen(fake_list, real_list, batch_size, frame_threshold,type_):

    '''fake_list : list of paths of the videos in the fake data directory
        real_list : list of paths of the videos in the real data directory
        batch_size : integer, number of videos to be fetch
        frame_threshold : integer, number of frames to keep in all videos. Padding will be done automatically
        
        Return: return a tuple (data, labels)'''

    num_videos = len(fake_list)
    real_data_index = np.random.permutation(np.arange(0,num_videos))
    fake_data_index = np.random.permutation(np.arange(0,num_videos))
    count = 0
    
    rnet = keras.applications.ResNet50(include_top = False, weights = 'imagenet')
    preprocess = keras.applications.resnet.preprocess_input
        
    while True:

        fake_data_array = np.zeros(shape = (batch_size//2,frame_threshold,7,7,2048))
        real_data_array = np.zeros(shape = (batch_size//2, frame_threshold, 7,7,2048))

        fake_labels = np.ones(shape = batch_size//2, dtype = int)
        real_labels = np.zeros(shape = batch_size//2, dtype = int)
        
        for i in range(batch_size//2):
            

            fake_array = np.load(fake_list[fake_data_index[count]])['arr_0']
            real_array = np.load(real_list[real_data_index[count]])['arr_0']


            # Keeping the number of frames to be same by either slicing the video or adding padding to the video

            if fake_array.shape[0] < frame_threshold: # Padding to fake video
                remaining_frames = frame_threshold - fake_array.shape[0]             
                fake_array = np.concatenate([fake_array,np.zeros(shape = (remaining_frames,224,224,3))], axis = 0)

            if fake_array.shape[0] > frame_threshold: # slicing the fake video
                fake_array = fake_array[:frame_threshold]

            if real_array.shape[0] < frame_threshold: # Padding to real video
                remaining_frames = frame_threshold - real_array.shape[0]
                real_array = np.concatenate([real_array,np.zeros(shape = (remaining_frames,224,224,3))], axis = 0)

            if real_array.shape[0] > frame_threshold: # slicing the real video
                real_array = real_array[:frame_threshold]

            fake_data_array[i] = rnet.predict(preprocess(fake_array), verbose = False)
            real_data_array[i] = rnet.predict(preprocess(real_array),verbose = False)
                

            count = count + 1
        
            if abs(count - real_data_index.shape[0]) < batch_size:
                
                real_data_index = np.random.permutation(np.arange(0,num_videos))
                fake_data_index = np.random.permutation(np.arange(0,num_videos))
                
                count = 0

        data_array = np.concatenate([fake_data_array, real_data_array], axis = 0)
        label_array = np.concatenate([fake_labels, real_labels])
        # index = np.random.permutation(range(batch_size))
        yield (data_array, label_array)


In [6]:
import gc

In [3]:
import gc
def data_gen(fake_list, real_list, batch_size, frame_threshold,type_, shuffle = True):

    '''fake_list : list of paths of the videos in the fake data directory
        real_list : list of paths of the videos in the real data directory
        batch_size : integer, number of videos to be fetch. When using it for validation gen, then choose the size such that it compleatly divides the total number of videos in one validation set
        frame_threshold : integer, number of frames to keep in all videos. Padding will be done automatically
        shuffle : Default (Shuffle = True), Used to shuffle the dataset in each epoch. For the validation generator, shuffle == False, provides better generalization results.
        
        Return: return a tuple (data, labels)'''
    
    mix_data = np.concatenate((fake_list, real_list))
    labels = np.concatenate((np.zeros(shape=(fake_list.shape[0])), np.ones(shape = (real_list.shape[0])))).astype(np.float32)
    num_videos = len(fake_list) * 2
    mix_data_index = np.random.permutation(np.arange(0,num_videos))
    count = 0
    
    rnet = keras.applications.ResNet50(include_top = False, weights = 'imagenet')
    preprocess = keras.applications.resnet.preprocess_input
        
    while True:
        mix_data_array = tf.Variable(tf.zeros(shape = (batch_size,frame_threshold,7,7,2048)))
        mix_labels =  tf.Variable(tf.zeros(shape = (batch_size,)))

        for i in range(batch_size):

            mix_array = np.load(mix_data[mix_data_index[count]])['arr_0']



            if mix_array.shape[0] < frame_threshold: # Adding reaming frames to video
                remaining_frames = frame_threshold - mix_array.shape[0]             
                mix_array = np.concatenate([mix_array,np.zeros(shape = (remaining_frames,224,224,3))], axis = 0)

            if mix_array.shape[0] > frame_threshold: # Slicing the extra frames from the video
                mix_array = mix_array[:frame_threshold]


            mix_data_array[i].assign(tf.constant(rnet.predict(preprocess(mix_array), verbose = False)))
            mix_labels[i].assign(tf.constant(labels[mix_data_index[count]]))
                

            count = count + 1
        
        if (abs(count - mix_data_index.shape[0]) < batch_size) & (shuffle == False):
            
            count = 0
            gc.collect()

        elif (abs(count - mix_data_index.shape[0]) < batch_size) & (shuffle == True):
            
            mix_data_index = np.random.permutation(np.arange(0,num_videos))
            count = 0
            gc.collect()

                

        yield (mix_data_array, mix_labels)


In [4]:
fake_data_array = np.array([], dtype = 'object')
for i,j,k in os.walk('/kaggle/input/timit-fake-real-dataset-numpy/Fake_Data_Numpy'):
    temp = i + '/' + np.array(k, dtype = 'object')
    fake_data_array = np.append(fake_data_array,temp)

In [5]:
real_data_array = np.array([], dtype = 'object')
for i,j,k in os.walk('/kaggle/input/timit-fake-real-dataset-numpy/Real_Data_Numpy/'):
    temp = i + '/' + np.array(k, dtype = 'object')
    real_data_array = np.append(real_data_array,temp)

In [6]:
batch_size = 4
n_frame = 80

In [7]:
l2 = keras.regularizers.l2

In [8]:
training_size = int(fake_data_array.shape[0]  * 0.8)
validation_size = int(fake_data_array.shape[0] * 0.2)

In [9]:
training_gen = data_gen(fake_data_array[:training_size], real_data_array[:training_size],batch_size,n_frame,type_='training')
validation_gen = data_gen(fake_data_array[training_size : training_size + validation_size], real_data_array[training_size : training_size + validation_size], batch_size, n_frame, type_ = 'val',shuffle =False )

In [9]:
def network():
    input_ = keras.layers.Input(shape = (n_frame, 7,7,2048))
    
    x = keras.layers.TimeDistributed(keras.layers.BatchNormalization())(input_)
    x = keras.layers.TimeDistributed(keras.layers.Flatten())(x)
    
    x = keras.layers.LSTM(64, kernel_regularizer=l2(0.1))(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(units = 1,activation='sigmoid')(x) 
    
    return keras.models.Model(input_, x)


In [10]:
model = network()

In [11]:
adam = keras.optimizers.Adam(learning_rate=10**(-5))

In [12]:
model_c = keras.callbacks.ModelCheckpoint("/kaggle/working/model_final.keras", monitor = 'val_accuracy', save_best_only = True)
model = keras.models.load_model("/kaggle/working/model_final.keras")
model.compile(optimizer=adam, loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(training_gen, epochs=10, steps_per_epoch=(training_size* 2)//batch_size, validation_data= validation_gen, validation_steps = 32, callbacks = [model_c])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 441s 3s/step - accuracy: 0.9074 - loss: 8.0405 - val_accuracy: 0.8281 - val_loss: 6.8366
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 346s 3s/step - accuracy: 0.9104 - loss: 6.3627 - val_accuracy: 0.8828 - val_loss: 5.7691
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 336s 3s/step - accuracy: 0.9330 - loss: 5.5020 - val_accuracy: 0.8750 - val_loss: 5.1768
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 321s 3s/step - accuracy: 0.9419 - loss: 4.8935 - val_accuracy: 0.7812 - val_loss: 4.8201
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.9433 - loss: 4.4649 - val_accuracy: 0.8438 - val_loss: 4.4107
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.9547 - loss: 4.0536 - val_accuracy: 0.8906 - val_loss: 4.0075
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.9300 - loss: 3.8442 - val_accuracy: 0.8516 - val_loss: 3.7646
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 293s 2

In [ ]:
model.save("/kaggle/working/model_2.keras")